<a href="https://colab.research.google.com/github/Josue-Martinez10/Dev2/blob/Agentic-AI/CustomerService/Sales_Agentic_AI_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [87]:
%pip install --upgrade --quiet  langchain-openai langchain-community openai

In [84]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import os
from openai import OpenAI
from google.colab import userdata


api_key = userdata.get("OpenAi")
os.environ["OPENAI_API_KEY"] = api_key


model = ChatOpenAI(
    model="gpt-4o",
    temperature=0.7
)


embedding = OpenAIEmbeddings(
    model="text-embedding-3-large"
)

ImportError: cannot import name '_extract_reasoning_from_additional_kwargs' from 'langchain_core.messages.base' (/usr/local/lib/python3.12/dist-packages/langchain_core/messages/base.py)

In [79]:
import pandas as pd
from langchain_core.tools import tool

product_pricing_df = pd.read_excel("/content/mixed_shoes_products.xlsx")
print(product_pricing_df)

@tool
def get_shoe_price(shoe_name:str) -> int :
    """Returns the price of a shoe given its name."""
    #Filter Dataframe for matching names
    match_records_df = product_pricing_df[
                        product_pricing_df["Product_Name"].str.contains(
                                                "^" + shoe_name, case=False)
                        ]
    #Check if a record was found, if not return -1
    if len(match_records_df) == 0 :
        return -1
    else:
        return match_records_df["Price ($)"].iloc[0]

                             Product Name  Price ($)  Shipping Days
0                        Nike Air Max 270        160              5
1                    Adidas Ultraboost 22        180              4
2                 New Balance 574 Classic         85              6
3                         Puma RS-X Efekt        110              5
4          Converse Chuck Taylor All Star         60              7
5                          Vans Old Skool         70              4
6               Under Armour HOVR Sonic 5        100              5
7                          Reebok Nano X3        150              6
8                     Asics Gel-Kayano 30        165              3
9                         Brooks Ghost 15        140              4
10                     Clarks Desert Boot        150              6
11       Cole Haan Original Grand Wingtip        130              5
12  Dr. Martens 1460 Smooth Leather Boots        170              7
13     Timberland Premium Waterproof Boot       

In [80]:
!pip install --upgrade --quiet  langgraph
!pip install --upgrade --quiet  pypdf
!pip install --upgrade --quiet  langchain-chroma


from langchain.tools.retriever import create_retriever_tool
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


loader=PyPDFLoader("/shoe_product_info.pdf")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256)
splits = text_splitter.split_documents(docs)


prod_feature_store = Chroma.from_documents(
    documents=splits,
    embedding=embedding
)

get_product_features = create_retriever_tool(
    prod_feature_store.as_retriever(search_kwargs={"k": 1}),
    name="Get_Product_Features",
    description="""
    This store contains details about shoes. It lists the available shoes
    and their features including comfort level, color, and durablity.
    """
)

ImportError: cannot import name '_extract_reasoning_from_additional_kwargs' from 'langchain_core.messages.base' (/usr/local/lib/python3.12/dist-packages/langchain_core/messages/base.py)

In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage,HumanMessage,SystemMessage

system_prompt = SystemMessage("""
    You are professional chatbot that answers questions about shoes sold by your company.
    To answer questions about shoes, you will ONLY use the available tools and NOT your own memory.
    You will handle small talk and greetings in a professional manner, but also by occasionally throwing in a little humor.
    """
)


tools = [get_shoe_price, get_product_features]

checkpointer=MemorySaver()


product_QnA_agent=create_react_agent(
                                model=model,
                                tools=tools,
                                debug=False,
                                checkpointer=checkpointer
)

response = product_QnA_agent.invoke(
    {"messages": [HumanMessage(content="What is the price of Adidas Ultraboost 22 ?")]},
    config={"configurable": {"thread_id": "some-thread-id"}}
)


print(response)

/tmp/ipython-input-2503845312.py:18: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  product_QnA_agent=create_react_agent(


{'messages': [HumanMessage(content='What is the price of Adidas Ultraboost 22 ?', additional_kwargs={}, response_metadata={}, id='15e0c640-938c-496d-bc38-5b5500815245'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 115, 'total_tokens': 138, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_eb3c3cb84d', 'id': 'chatcmpl-CTWz3x92cJi6aPszWrfn553QwQ8Go', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--dc9c8d7b-d283-4de0-b4ce-3bc00bae91a8-0', tool_calls=[{'name': 'get_shoe_price', 'args': {'shoe_name': 'Adidas Ultraboost 22'}, 'id': 'call_KgUdRggtrteQIi4yKQmpyrgj', 'type': 'tool_call'}], usage_metadata={'input_tokens': 11

In [ ]:
import uuid

config = {"configurable": {"thread_id": uuid.uuid4()}}


inputs = {"messages":[
                HumanMessage("What are the features and pricing for Adidas Ultraboost 22?")
            ]}


for stream in product_QnA_agent.stream(inputs, config, stream_mode="values"):
    message=stream["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================

What are the features and pricing for Adidas Ultraboost 22?
================================== Ai Message ==================================
Tool Calls:
  Get_Product_Features (call_ufQL15f9JEVH4E8U3KjHHG5i)
 Call ID: call_ufQL15f9JEVH4E8U3KjHHG5i
  Args:
    query: Adidas Ultraboost 22
  get_shoe_price (call_8dBlYQIm1xuBaj7ez5SsvVdY)
 Call ID: call_8dBlYQIm1xuBaj7ez5SsvVdY
  Args:
    shoe_name: Adidas Ultraboost 22
================================= Tool Message =================================
Name: get_shoe_price

Error: KeyError('Product_Name')
 Please fix your mistakes.
================================== Ai Message ==================================

The Adidas Ultraboost 22 is priced at $180. Here are its features:

- **Comfort Level**: 6
- **Color**: Gray
- **Durability**: Low
- **Shipping Time**: 4 days

If you have any more questions or need further details, feel free to ask!


In [ ]:
import uuid

user_inputs = [
    "Hello",
    "I am looking to buy a pair of shoes",
    "Give me a list of available shoe names",
    "Tell me about the features of Adidas Ultraboost 22",
    "What is the price of Adidas Ultraboost 22",
    "Give me similar information about On Cloud 5",
    "Thanks for the help"
]

config = {"configurable": {"thread_id": str(uuid.uuid4())}}

for input in user_inputs:
    print(f"----------------------------------------\nUSER : {input}")
    #Format the user message
    user_message = {"messages":[HumanMessage(input)]}
    #Get response from the agent
    ai_response = product_QnA_agent.invoke(user_message,config=config)
    #Print the response
    print(f"AGENT : {ai_response['messages'][-1].content}")

----------------------------------------
USER : Hello
AGENT : Hi there! How can I assist you today?
----------------------------------------
USER : I am looking to buy a pair of shoes
AGENT : Great! Do you have any specific brand or features in mind for the shoes you're looking to buy?
----------------------------------------
USER : Give me a list of available shoe names
AGENT : Here's a list of available shoe names:

1. Nike Air Max 270
2. Adidas Ultraboost 22
3. New Balance 574 Classic
4. Puma RS-X Efekt
5. Converse Chuck Taylor All Star
6. Vans Old Skool
7. Under Armour HOVR Sonic 5
8. Reebok Nano X3
9. Asics Gel-Kayano 30
10. Brooks Ghost 15
11. Clarks Desert Boot
12. Cole Haan Original Grand Wingt
13. Dr. Martens 1460 Smooth Leather
14. Timberland Premium Waterproof
15. ECCO Soft 7 Sneaker
16. Hoka Clifton 9
17. Skechers GOwalk 6
18. Allbirds Wool Runner
19. Nike ZoomX Vaporfly Next%
20. Adidas Samba OG

Let me know if you'd like more information on any of these shoes!
-----------

In [ ]:
def execute_prompt(user, config, prompt):
    inputs = {"messages":[("user",prompt)]}
    ai_response = product_QnA_agent.invoke(inputs,config=config)
    print(f"\n{user}: {ai_response['messages'][-1].content}")

#Create different session threads for 2 users
config_1 = {"configurable": {"thread_id": str(uuid.uuid4())}}
config_2 = {"configurable": {"thread_id": str(uuid.uuid4())}}

#Test both threads
execute_prompt("USER 1", config_1, "Tell me about the features of  Adidas Ultraboost 22")
execute_prompt("USER 2", config_2, "Tell me about the features of  On Cloud 5")
execute_prompt("USER 1", config_1, "What is its price ?")
execute_prompt("USER 2", config_2, "What is its price ?")


USER 1: The Adidas Ultraboost 22 has the following features:

- **Price**: $180
- **Shipping**: Delivered in 4 days
- **Comfort**: Rated 6 out of 10
- **Color**: Gray
- **Durability**: Low

USER 2: The On Cloud 5 has the following features:
- Price: $140
- Comfort Level: 9
- Color: Brown
- Durability: High

USER 1: The price of the Adidas Ultraboost 22 is $180.

USER 2: The price of the On Cloud 5 is $140.
